In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Game Results

In [3]:
# load games dataset
df_games = pd.read_csv('../data/games/all_games.csv')

In [4]:
# Select season to create game results for
season_selection = list(range(2008, 2022))

In [5]:
# create raw game results
game_results_seasons = []
for i in range(len(season_selection)):
    single_season_games = df_games[['gameId', 'gameDate', 'season', 'playerTeam', 'opposingTeam', 'home_or_away', 'goalsFor', 'goalsAgainst']] \
        .loc[(df_games['season'] == season_selection[i]) & (df_games['situation'] == 'all') & (df_games['playoffGame'] == 0)] \
        .drop_duplicates(subset='gameId') \
        .drop(['gameId'], axis=1)
    single_season_games[['playerTeam', 'opposingTeam']] = single_season_games[['playerTeam', 'opposingTeam']].apply(lambda x: x.str.replace('.', ''))
    game_results_seasons.append(single_season_games)
game_results = pd.concat(game_results_seasons)

In [6]:
# define fucntion to create column of game results
# 2 is a win for 'playerTeam'
# 1 is a win for 'opposingTeam'
# 0 is a game that went to shootout
def game_result_label_race(row):
    if row['goalsFor'] > row['goalsAgainst']:
        return 'team 1'
    if row['goalsFor'] < row['goalsAgainst']:
        return 'team 2'
    if row['goalsFor'] == row['goalsAgainst']:
        return 'shootout'

# define fucntion that encodes home team
# 1 if team1 is home team
# 2 if team2 is home team
def home_team_t1_label_race(row):
    if row['home_or_away'] == 'HOME':
        return 1
    if row['home_or_away'] == 'AWAY':
        return 0
    
    
def home_team_t2_label_race(row):
    if row['home_or_away'] == 'HOME':
        return 0
    if row['home_or_away'] == 'AWAY':
        return 1   

In [7]:
game_results['home_or_away_t1'] = np.nan
game_results['home_or_away_t2'] = np.nan

# apply functions to crate column of game results and encode home team
game_results['result'] = game_results.apply(lambda row: game_result_label_race(row), axis=1)
game_results['home_or_away_t1'] = game_results.apply(lambda row: home_team_t1_label_race(row), axis=1)
game_results['home_or_away_t2'] = game_results.apply(lambda row: home_team_t2_label_race(row), axis=1)

In [8]:
# rename, drop and order columns for usability + reindexing
game_results.rename(columns={'playerTeam': 'team1', 'opposingTeam': 'team2'}, inplace=True)
game_results.drop(['goalsFor', 'goalsAgainst', 'home_or_away'], axis=1, inplace=True)
game_results = game_results[['gameDate', 'season', 'team1', 'team2', 'result', 'home_or_away_t1', 'home_or_away_t2']].sort_values('gameDate').reset_index(drop=True)

# convert date to python date
game_results['gameDate'] = pd.to_datetime(game_results['gameDate'], format = '%Y%m%d')

In [9]:
game_results[game_results.isnull().any(axis=1)]

,gameDate,season,team1,team2,result,home_or_away_t1,home_or_away_t2


## Team Stats

### Aggregate Stats

In [10]:
# load team stats datasets
team_stats_seasons = [pd.read_csv('../data/team_stats/teams_08.csv'),
                      pd.read_csv('../data/team_stats/teams_09.csv'),
                      pd.read_csv('../data/team_stats/teams_10.csv'),
                      pd.read_csv('../data/team_stats/teams_11.csv'),
                      pd.read_csv('../data/team_stats/teams_12.csv'),
                      pd.read_csv('../data/team_stats/teams_13.csv'),
                      pd.read_csv('../data/team_stats/teams_14.csv'),
                      pd.read_csv('../data/team_stats/teams_15.csv'),
                      pd.read_csv('../data/team_stats/teams_16.csv'),
                      pd.read_csv('../data/team_stats/teams_17.csv'),
                      pd.read_csv('../data/team_stats/teams_18.csv'),
                      pd.read_csv('../data/team_stats/teams_19.csv'),
                      pd.read_csv('../data/team_stats/teams_20.csv'),
                      pd.read_csv('../data/team_stats/teams_21.csv'),
]

df_teams = pd.concat(team_stats_seasons)

In [11]:
# select team stats to include in modelProgram
tracked_statistics = ['name', 'season',
                      'xGoalsPercentage',
                      'flurryScoreVenueAdjustedxGoalsFor', 'penaltiesFor', 'dZoneGiveawaysAgainst',
                      'flurryScoreVenueAdjustedxGoalsAgainst', 'penaltiesAgainst', 'dZoneGiveawaysFor']

In [12]:
# isolate team stats from all situations and drop columns for usability 
team_stats_5on5 = df_teams[tracked_statistics].loc[(df_teams['situation'] == '5on5')].add_suffix('_5on5').rename(columns={"name_5on5": "name", "season_5on5": "season"})
team_stats_5on4 = df_teams[tracked_statistics].loc[(df_teams['situation'] == '5on4')].add_suffix('_5on4').rename(columns={"name_5on4": "name", "season_5on4": "season"})
team_stats_4on5 = df_teams[tracked_statistics].loc[(df_teams['situation'] == '4on5')].add_suffix('_4on5').rename(columns={"name_4on5": "name", "season_4on5": "season"})

In [13]:
team_stats_5on5['team_strength_pos'] = team_stats_5on5.loc[:,'flurryScoreVenueAdjustedxGoalsFor_5on5':'dZoneGiveawaysAgainst_5on5'].mean(axis = 1)
team_stats_5on5['team_strength_neg'] = team_stats_5on5.loc[:,'flurryScoreVenueAdjustedxGoalsAgainst_5on5':].mean(axis = 1).mul(-.5)
team_stats_5on5['team_strength_5on5'] = team_stats_5on5.iloc[:, -2:].sum(axis=1)
team_stats_5on5.drop(['team_strength_pos', 'team_strength_neg'], axis=1, inplace=True)

team_stats_5on4['team_strength_pos'] = team_stats_5on4.loc[:,'flurryScoreVenueAdjustedxGoalsFor_5on4':'dZoneGiveawaysAgainst_5on4'].mean(axis = 1)
team_stats_5on4['team_strength_neg'] = team_stats_5on4.loc[:,'flurryScoreVenueAdjustedxGoalsAgainst_5on4':].mean(axis = 1).mul(-.5)
team_stats_5on4['team_strength_5on4'] = team_stats_5on4.iloc[:, -2:].sum(axis=1)
team_stats_5on4.drop(['team_strength_pos', 'team_strength_neg', 'xGoalsPercentage_5on4'], axis=1, inplace=True)

team_stats_4on5['team_strength_pos'] = team_stats_4on5.loc[:,'flurryScoreVenueAdjustedxGoalsFor_4on5':'dZoneGiveawaysAgainst_4on5'].mean(axis = 1)
team_stats_4on5['team_strength_neg'] = team_stats_4on5.loc[:,'flurryScoreVenueAdjustedxGoalsAgainst_4on5':].mean(axis = 1).mul(-.5)
team_stats_4on5['team_strength_4on5'] = team_stats_4on5.iloc[:, -2:].sum(axis=1)
team_stats_4on5.drop(['team_strength_pos', 'team_strength_neg', 'xGoalsPercentage_4on5'], axis=1, inplace=True)

In [14]:
# merge different situations into single dataframe
team_stats = pd.merge(team_stats_5on5, team_stats_5on4, how='left', on=['name', 'season'])
team_stats = pd.merge(team_stats, team_stats_4on5, how='left', on=['name', 'season'])

# clean team names
team_stats['name'] = team_stats['name'].str.replace('.', '')

In [15]:
team_stats.fillna(team_stats.median(), inplace=True)

In [16]:
team_stats = team_stats[['name', 'season', 'xGoalsPercentage_5on5', 'team_strength_5on5', 'team_strength_5on4', 'team_strength_4on5']]
team_stats[team_stats.isnull().any(axis=1)]

,name,season,xGoalsPercentage_5on5,team_strength_5on5,team_strength_5on4,team_strength_4on5


### Time Series Stats

In [17]:
ts = pd.read_csv('../data/games/all_games.csv')

In [18]:
ts.sort_values('gameDate', inplace=True)

ts = ts.loc[(ts['situation'] == '5on5') & (ts['playoffGame'] == 0)]
ts[['team']] = ts[['team']].apply(lambda x: x.str.replace('.', ''))

ts['gameDate'] =  pd.to_datetime(ts['gameDate'], format = '%Y%m%d')

ts = ts[['team', 'gameDate', 'xGoalsPercentage']]

teams_list = list(ts['team'].unique())

In [19]:
timeseries_teams = {}

for i  in range(len(teams_list)):
    timeseries_teams[f'{teams_list[i]}'] = ts.loc[ts['team'] == teams_list[i]]

In [20]:
for i in teams_list:
    df_ts_team = timeseries_teams[i].copy()
    df_ts_team['xGoalsPercentage_last_3'] = df_ts_team['xGoalsPercentage'].rolling(window=3, closed= "left").mean().fillna(.5)
    df_ts_team['xGoalsPercentage_last_5'] = df_ts_team['xGoalsPercentage'].rolling(window=5, closed= "left").mean().fillna(.5)
    df_ts_team['xGoalsPercentage_last_10'] = df_ts_team['xGoalsPercentage'].rolling(window=10, closed= "left").mean().fillna(.5)
    timeseries_teams[i] = df_ts_team

In [21]:
ts_stats = pd.concat(timeseries_teams.values(), ignore_index=True).drop(['xGoalsPercentage'], axis=1)
ts_stats[ts_stats.isnull().any(axis=1)]

,team,gameDate,xGoalsPercentage_last_3,xGoalsPercentage_last_5,xGoalsPercentage_last_10


## Goalies

In [22]:
df_goalies_labels = pd.read_csv('../data/goalie_stats/goalies_20.csv')
goalies_21 = pd.read_csv('../data/goalie_stats/goalies_21.csv', header=None)
goalies_21 = pd.DataFrame(data=goalies_21.values, columns=df_goalies_labels.columns)

In [23]:
# load goalie stats datasets
goalie_stats_seasons = [pd.read_csv('../data/goalie_stats/goalies_08.csv'),
                        pd.read_csv('../data/goalie_stats/goalies_09.csv'),
                        pd.read_csv('../data/goalie_stats/goalies_10.csv'),
                        pd.read_csv('../data/goalie_stats/goalies_11.csv'),
                        pd.read_csv('../data/goalie_stats/goalies_12.csv'),
                        pd.read_csv('../data/goalie_stats/goalies_13.csv'),
                        pd.read_csv('../data/goalie_stats/goalies_14.csv'),
                        pd.read_csv('../data/goalie_stats/goalies_15.csv'),
                        pd.read_csv('../data/goalie_stats/goalies_16.csv'),
                        pd.read_csv('../data/goalie_stats/goalies_17.csv'),
                        pd.read_csv('../data/goalie_stats/goalies_18.csv'),
                        pd.read_csv('../data/goalie_stats/goalies_19.csv'),
                        pd.read_csv('../data/goalie_stats/goalies_20.csv'),
                        goalies_21]

df_goalies = pd.concat(goalie_stats_seasons)

In [24]:
# select team stats to include in model
tracked_statistics = ['name', 'season', 'team', 'games_played',
                      'xRebounds', 'xFreeze', 'xOnGoal',
                      'xPlayStopped','xPlayContinuedInZone', 'xPlayContinuedOutsideZone',  
                      'lowDangerxGoals', 'mediumDangerxGoals', 'highDangerxGoals']

# select stats to regularize by games played
goalie_reg = tracked_statistics[4:]

In [25]:
# isolate gaolie stats from all situations and drop columns for usability 
goalie_stats_indv = df_goalies[tracked_statistics].loc[(df_goalies['situation'] == '5on5')].drop(['name'], axis=1)

# clean team names
goalie_stats_indv['team'] = goalie_stats_indv['team'].str.replace('.', '')

In [26]:
# regularize goalie stats to games played
# proportion stats to games played
goalie_stats_indv[goalie_reg] = goalie_stats_indv[goalie_reg].multiply(goalie_stats_indv['games_played'], axis='index').divide(82, axis='index')
# group by team and season
goalie_stats = goalie_stats_indv.groupby(by=['team', 'season']).sum().reset_index()
# regularize games played to 82 game season and drop games_played
goalie_stats[goalie_reg] = goalie_stats[goalie_reg].divide(goalie_stats['games_played'], axis='index').multiply(82, axis='index')
goalie_stats.drop(['games_played'], axis=1, inplace=True)

In [27]:
# scale goalie stats for wieghting
scaler = MinMaxScaler()
goalie_stats[goalie_reg] = scaler.fit_transform(goalie_stats[goalie_reg])

# weight minor stats
goalie_weighting = goalie_reg[:6]
goalie_stats[goalie_weighting] = goalie_stats[goalie_weighting] * 1
# weight major stats
goalie_weighting = goalie_reg[6:]
goalie_stats[goalie_weighting] = goalie_stats[goalie_weighting] * 2

# create goalie metric and drop old stats
goalie_stats['goalie_strength'] = goalie_stats.loc[:,'xRebounds':].mean(axis = 1)
goalie_stats.drop(goalie_reg, axis=1, inplace=True)

goalie_stats[goalie_stats.isnull().any(axis=1)]

,team,season,goalie_strength


## Lines/Pairings

In [28]:
# load goalie stats datasets
line_stats_seasons = [pd.read_csv('../data/line_stats/lines_08.csv'),
                       pd.read_csv('../data/line_stats/lines_09.csv'),
                       pd.read_csv('../data/line_stats/lines_10.csv'),
                       pd.read_csv('../data/line_stats/lines_11.csv'),
                       pd.read_csv('../data/line_stats/lines_12.csv'),
                       pd.read_csv('../data/line_stats/lines_13.csv'),
                       pd.read_csv('../data/line_stats/lines_14.csv'),
                       pd.read_csv('../data/line_stats/lines_15.csv'),
                       pd.read_csv('../data/line_stats/lines_16.csv'),
                       pd.read_csv('../data/line_stats/lines_17.csv'),
                       pd.read_csv('../data/line_stats/lines_18.csv'),
                       pd.read_csv('../data/line_stats/lines_19.csv'),
                       pd.read_csv('../data/line_stats/lines_20.csv'),
                       pd.read_csv('../data/line_stats/lines_21.csv'),
]

df_lines = pd.concat(line_stats_seasons)

In [29]:
# select team stats to include in model
tracked_statistics = ['position', 'season', 'team', 'icetime',
                      'flurryScoreVenueAdjustedxGoalsFor', 'xOnGoalFor', 'xReboundsFor', 'reboundxGoalsFor',
                      'xPlayStoppedFor', 'xPlayContinuedInZoneFor', 'xPlayContinuedOutsideZoneFor',
                      'penaltiesAgainst', 'dZoneGiveawaysAgainst', 'takeawaysFor',
                      'lowDangerxGoalsFor', 'mediumDangerxGoalsFor','highDangerxGoalsFor',
                      'flurryScoreVenueAdjustedxGoalsAgainst', 'xOnGoalAgainst', 'xReboundsAgainst', 'reboundxGoalsAgainst',
                      'xPlayStoppedAgainst', 'xPlayContinuedInZoneAgainst', 'xPlayContinuedOutsideZoneAgainst',
                      'penaltiesFor', 'dZoneGiveawaysFor', 'takeawaysAgainst',
                      'lowDangerxGoalsAgainst', 'mediumDangerxGoalsAgainst','highDangerxGoalsAgainst']

# select stats to regularize by games played
lines_reg = tracked_statistics[4:]

In [30]:
# clean team names
df_lines['team'] = df_lines['team'].str.replace('.', '')

# isolate line and pairing stats from each other 
line_stats = df_lines[tracked_statistics].loc[(df_lines['position'] == 'line')].drop(['position'], axis=1).reset_index(drop=True)
pairing_stats = df_lines[tracked_statistics].loc[(df_lines['position'] == 'pairing')].drop(['position'], axis=1).reset_index(drop=True)

In [31]:
len(line_stats)

129056

In [32]:
# select 4 top lines by icetime for each team for each season to be used as lines 1-4
line_stats = line_stats.iloc[line_stats.groupby(['season', 'team'], sort=False)['icetime'].nlargest(4).index.get_level_values(2).to_list()]

# select 3 top pairings by icetime for each team for each season to be used as pairings 1-3
pairing_stats = pairing_stats.iloc[pairing_stats.groupby(['season', 'team'], sort=False)['icetime'].nlargest(3).index.get_level_values(2).to_list()]

# overwrite icetime with respective line numbers and rename column
line_stats['line'] = [1, 2, 3, 4] * 426
pairing_stats['line'] = [1, 2, 3] * 426

### Line 1

In [33]:
# pull lines
first_lines = line_stats.loc[(line_stats['line'] == 1)]
second_lines = line_stats.loc[(line_stats['line'] == 2)]
third_lines = line_stats.loc[(line_stats['line'] == 3)]
fourth_lines = line_stats.loc[(line_stats['line'] == 4)]

# pull pairings
first_pairings = line_stats.loc[(line_stats['line'] == 1)]
second_pairings = line_stats.loc[(line_stats['line'] == 2)]
third_pairings = line_stats.loc[(line_stats['line'] == 3)]

In [34]:
# regularize line stats to average seconds played per line
first_lines[lines_reg] = first_lines[lines_reg].divide(first_lines['icetime'], axis='index').multiply(50000, axis='index')
second_lines[lines_reg] = second_lines[lines_reg].divide(second_lines['icetime'], axis='index').multiply(50000, axis='index')
third_lines[lines_reg] = third_lines[lines_reg].divide(third_lines['icetime'], axis='index').multiply(50000, axis='index')
fourth_lines[lines_reg] = fourth_lines[lines_reg].divide(fourth_lines ['icetime'], axis='index').multiply(50000, axis='index')

# regularize pairing stats to average seconds played per pairing
first_pairings[lines_reg] = first_pairings[lines_reg].divide(first_pairings['icetime'], axis='index').multiply(50000, axis='index')
second_pairings[lines_reg] = second_pairings[lines_reg].divide(second_pairings['icetime'], axis='index').multiply(50000, axis='index')
third_pairings[lines_reg] = third_pairings[lines_reg].divide(third_pairings['icetime'], axis='index').multiply(50000, axis='index')

# drop icetime column in line dfs
first_lines.drop(['icetime'], axis=1, inplace=True)
second_lines.drop(['icetime'], axis=1, inplace=True)
third_lines.drop(['icetime'], axis=1, inplace=True)
fourth_lines.drop(['icetime'], axis=1, inplace=True)
first_pairings.drop(['icetime'], axis=1, inplace=True)
second_pairings.drop(['icetime'], axis=1, inplace=True)
third_pairings.drop(['icetime'], axis=1, inplace=True)

In [35]:
# scale goalie stats for wieghting
scaler = MinMaxScaler()

first_lines[lines_reg] = scaler.fit_transform(first_lines[lines_reg])
second_lines[lines_reg] = scaler.fit_transform(second_lines[lines_reg])
third_lines[lines_reg] = scaler.fit_transform(third_lines[lines_reg])
fourth_lines[lines_reg] = scaler.fit_transform(fourth_lines[lines_reg])

first_pairings[lines_reg] = scaler.fit_transform(first_pairings[lines_reg])
second_pairings[lines_reg] = scaler.fit_transform(second_pairings[lines_reg])
third_pairings[lines_reg] = scaler.fit_transform(third_pairings[lines_reg])

In [36]:
major_stats = ['flurryScoreVenueAdjustedxGoalsFor', 'flurryScoreVenueAdjustedxGoalsAgainst', 
               'penaltiesFor', 'penaltiesAgainst',
               'takeawaysFor', 'takeawaysAgainst',
               'dZoneGiveawaysFor', 'dZoneGiveawaysAgainst',
               'mediumDangerxGoalsFor', 'mediumDangerxGoalsAgainst',
               'highDangerxGoalsFor', 'highDangerxGoalsAgainst']

minor_stats = ['xOnGoalFor', 'xOnGoalAgainst', 
               'xReboundsFor', 'xReboundsAgainst', 
               'xPlayStoppedFor', 'xPlayStoppedAgainst', 
               'xPlayContinuedInZoneFor', 'xPlayContinuedInZoneAgainst',
               'xPlayContinuedOutsideZoneFor', 'xPlayContinuedOutsideZoneAgainst',
               'reboundxGoalsFor', 'reboundxGoalsAgainst',
               'lowDangerxGoalsFor', 'lowDangerxGoalsAgainst']

In [37]:
first_lines[major_stats] = first_lines[major_stats] * 2
first_lines[minor_stats] = first_lines[minor_stats] * 1

second_lines[major_stats] = second_lines[major_stats] * 2
second_lines[minor_stats] = second_lines[minor_stats] * 1

third_lines[major_stats] = third_lines[major_stats] * 2
third_lines[minor_stats] = third_lines[minor_stats] * 1

fourth_lines[major_stats] = fourth_lines[major_stats] * 2
fourth_lines[minor_stats] = fourth_lines[minor_stats] * 1

first_pairings[major_stats] = first_pairings[major_stats] * 2
first_pairings[minor_stats] = first_pairings[minor_stats] * 1

second_pairings[major_stats] = second_pairings[major_stats] * 2
second_pairings[minor_stats] = second_pairings[minor_stats] * 1

third_pairings[major_stats] = third_pairings[major_stats] * 2
third_pairings[minor_stats] = third_pairings[minor_stats] * 1

In [38]:
# create goalie metric and drop old stats
first_lines['line1_strength_pos'] = first_lines.loc[:,'flurryScoreVenueAdjustedxGoalsFor':'highDangerxGoalsFor'].mean(axis = 1)
first_lines['line1_strength_neg'] = first_lines.loc[:,'flurryScoreVenueAdjustedxGoalsAgainst':].mean(axis = 1).mul(-.5)
first_lines['line1_strength'] = first_lines.iloc[:, -2:].sum(axis=1)
first_lines.drop(['line1_strength_pos', 'line1_strength_neg', 'line'], axis=1, inplace=True)


second_lines['line2_strength_pos'] = second_lines.loc[:,'flurryScoreVenueAdjustedxGoalsFor':'highDangerxGoalsFor'].mean(axis = 1)
second_lines['line2_strength_neg'] = second_lines.loc[:,'flurryScoreVenueAdjustedxGoalsAgainst':].mean(axis = 1).mul(-.5)
second_lines['line2_strength'] = second_lines.iloc[:, -2:].sum(axis=1)
second_lines.drop(['line2_strength_pos', 'line2_strength_neg', 'line'], axis=1, inplace=True)

third_lines['line3_strength_pos'] = third_lines.loc[:,'flurryScoreVenueAdjustedxGoalsFor':'highDangerxGoalsFor'].mean(axis = 1)
third_lines['line3_strength_neg'] = third_lines.loc[:,'flurryScoreVenueAdjustedxGoalsAgainst':].mean(axis = 1).mul(-.5)
third_lines['line3_strength'] = third_lines.iloc[:, -2:].sum(axis=1)
third_lines.drop(['line3_strength_pos', 'line3_strength_neg', 'line'], axis=1, inplace=True)

fourth_lines['line4_strength_pos'] = fourth_lines.loc[:,'flurryScoreVenueAdjustedxGoalsFor':'highDangerxGoalsFor'].mean(axis = 1)
fourth_lines['line4_strength_neg'] = fourth_lines.loc[:,'flurryScoreVenueAdjustedxGoalsAgainst':].mean(axis = 1).mul(-.5)
fourth_lines['line4_strength'] = fourth_lines.iloc[:, -2:].sum(axis=1)
fourth_lines.drop(['line4_strength_pos', 'line4_strength_neg', 'line'], axis=1, inplace=True)

first_pairings['pair1_strength_pos'] = first_pairings.loc[:,'flurryScoreVenueAdjustedxGoalsFor':'highDangerxGoalsFor'].mean(axis = 1)
first_pairings['pair1_strength_neg'] = first_pairings.loc[:,'flurryScoreVenueAdjustedxGoalsAgainst':].mean(axis = 1).mul(-.1)
first_pairings['pair1_strength'] = first_pairings.iloc[:, -2:].sum(axis=1)
first_pairings.drop(['pair1_strength_pos', 'pair1_strength_neg', 'line'], axis=1, inplace=True)

second_pairings['pair2_strength_pos'] = second_pairings.loc[:,'flurryScoreVenueAdjustedxGoalsFor':'highDangerxGoalsFor'].mean(axis = 1)
second_pairings['pair2_strength_neg'] = second_pairings.loc[:,'flurryScoreVenueAdjustedxGoalsAgainst':].mean(axis = 1).mul(-.1)
second_pairings['pair2_strength'] = second_pairings.iloc[:, -2:].sum(axis=1)
second_pairings.drop(['pair2_strength_pos', 'pair2_strength_neg', 'line'], axis=1, inplace=True)

third_pairings['pair3_strength_pos'] = third_pairings.loc[:,'flurryScoreVenueAdjustedxGoalsFor':'highDangerxGoalsFor'].mean(axis = 1)
third_pairings['pair3_strength_neg'] = third_pairings.loc[:,'flurryScoreVenueAdjustedxGoalsAgainst':].mean(axis = 1).mul(-.1)
third_pairings['pair3_strength'] = third_pairings.iloc[:, -2:].sum(axis=1)
third_pairings.drop(['pair3_strength_pos', 'pair3_strength_neg', 'line'], axis=1, inplace=True)

In [39]:
# add line metrics to row for each team for each season
lp_stats = first_lines[['season', 'team']]
lp_stats['line1_strength'] = np.multiply(first_lines['line1_strength'].values, 32)
lp_stats['line2_strength'] = np.multiply(second_lines['line2_strength'].values, 27)
lp_stats['line3_strength'] = np.multiply(third_lines['line3_strength'].values, 22)
lp_stats['line4_strength'] = np.multiply(fourth_lines['line4_strength'].values, 19)
lp_stats['forward_strength'] = lp_stats.loc[:, 'line1_strength':'line4_strength'].sum(axis=1)

lp_stats['pair1_strength'] = np.multiply(first_pairings['pair1_strength'].values, 39)
lp_stats['pair2_strength'] = np.multiply(second_pairings['pair2_strength'].values, 33)
lp_stats['pair3_strength'] = np.multiply(third_pairings['pair3_strength'].values, 28)
lp_stats['defense_strength'] = lp_stats.loc[:, 'pair1_strength':'pair3_strength'].sum(axis=1)

lp_stats = lp_stats.drop(['line1_strength', 'line2_strength', 'line3_strength', 'line4_strength',
                          'pair1_strength', 'pair2_strength', 'pair3_strength'], axis=1).reset_index(drop=True)

lp_stats[lp_stats.isnull().any(axis=1)]

,season,team,forward_strength,defense_strength


In [40]:
import re

from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

In [41]:
df_games_web = pd.read_csv('../data/games/21-22_season.csv')

In [42]:
req = Request('https://moneypuck.com/moneypuck/playerData/seasonSummary/2021/regular/lines.csv')
req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
content = urlopen(req)

df_lines_web = pd.read_csv(content)

In [43]:
req = Request('https://moneypuck.com/moneypuck/playerData/seasonSummary/2021/regular/goalies.csv')
req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
content = urlopen(req)

df_goalies_web = pd.read_csv(content)

In [44]:
req = Request('https://moneypuck.com/moneypuck/playerData/seasonSummary/2021/regular/teams.csv')
req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
content = urlopen(req)

df_teams_web = pd.read_csv(content)

In [45]:
options = FirefoxOptions()
options.add_argument("--headless")
driver = webdriver.Firefox(options=options)

lineups = {}

teams = [x.lower() for x in ['Anaheim Ducks',
                             'Arizona Coyotes',
                             'Boston Bruins',
                             'Buffalo Sabres',
                             'Calgary Flames',
                             'Carolina Hurricanes',
                             'Chicago Blackhawks',
                             'Colorado Avalanche',
                             'Columbus Blue Jackets',
                             'Dallas Stars',
                             'Detroit Red Wings',
                             'Edmonton Oilers',
                             'Florida Panthers',
                             'Los Angeles Kings',
                             'Minnesota Wild',
                             'Montreal Canadiens',
                             'Nashville Predators',
                             'New Jersey Devils',
                             'New York Islanders',
                             'New York Rangers',
                             'Ottawa Senators',
                             'Philadelphia Flyers',
                             'Pittsburgh Penguins',
                             'San Jose Sharks',
                             'Seattle Kraken',
                             'St Louis Blues',
                             'Tampa Bay Lightning',
                             'Toronto Maple Leafs',
                             'Vancouver Canucks',
                             'Vegas Golden Knights',
                             'Washington Capitals',
                             'Winnipeg Jets']]

for i in range(len(teams)):
    teams[i] = teams[i].replace(" ", "-")

In [46]:
#for i in range(len(teams)):
#    driver.get(f"https://www.dailyfaceoff.com/teams/{teams[i]}/line-combinations/")
#
#    content = driver.page_source
#    soup = BeautifulSoup(content)
#
#    players = soup.findAll('span', attrs={'class':'player-name'})
#    players = [str(i) for i in players]
#
#    for j in range(len(players)):
#        s = players[j]
#        players[j] = re.search('<span class="player-name">(.*)</span>', s).group(1).split()[-1]
#        
#    players = players[:18] + players[-2:]
#    lines = [players[:3], players[3:6], players[6:9], players[9:12], players[12:14], players[14:16], players[16:18], players[-2], players[-1]]
#    
#    for k in range(len(lines) - 2):
#        lines[k] = '-'.join(lines[k])
#    
#    lineups[f'{teams[i]}'] = pd.DataFrame({'lines': lines}) 

In [47]:
df_games_web = df_games_web[['Date', 'Home', 'Visitor']]
today = datetime.today().strftime('%Y-%m-%d')
df_games_web.loc[df_games_web['Date'] == today]

## Zipping

In [50]:
# create copies of datasets with new names for columns to merge on
t1_team_stats = team_stats.rename(columns={'name': 'team1'})
t2_team_stats = team_stats.rename(columns={'name': 'team2'})

t1_goalies_stats = goalie_stats.rename(columns={'team': 'team1'})
t2_goalies_stats = goalie_stats.rename(columns={'team': 'team2'})

t1_lp_stats = lp_stats.rename(columns={'team': 'team1'})
t2_lp_stats = lp_stats.rename(columns={'team': 'team2'})

t1_ts_stats = ts_stats.rename(columns={'team': 'team1'})
t2_ts_stats = ts_stats.rename(columns={'team': 'team2'})

# merge datasets and game_results
input_df = pd.merge(game_results, t1_team_stats, how='left', on=['team1', 'season'])
input_df = pd.merge(input_df, t2_team_stats, how='left', on=['team2', 'season'], suffixes=('_1', '_2'))

input_df = pd.merge(input_df, t1_goalies_stats, how='left', on=['team1', 'season'])
input_df = pd.merge(input_df, t2_goalies_stats, how='left', on=['team2', 'season'], suffixes=('_1', '_2'))

input_df = pd.merge(input_df, t1_lp_stats, how='left', on=['team1', 'season'])
input_df = pd.merge(input_df, t2_lp_stats, how='left', on=['team2', 'season'], suffixes=('_1', '_2'))

input_df = pd.merge(input_df, t1_ts_stats, how='left', on=['team1', 'gameDate'])
input_df = pd.merge(input_df, t2_ts_stats, how='left', on=['team2', 'gameDate'], suffixes=('_1', '_2'))

# change teams to single column with tuples and set as index
input_df['teams'] = input_df[['team1', 'team2']].apply(tuple, axis=1)
input_df = input_df.set_index('gameDate').drop(['team1', 'team2', 'season'], axis=1)

In [51]:
input_df.to_csv('../data/output/input_df.csv')

In [52]:
input_df.shape

(15997, 24)

In [53]:
input_df.isnull().sum()

result                        0
home_or_away_t1               0
home_or_away_t2               0
xGoalsPercentage_5on5_1       0
team_strength_5on5_1          0
team_strength_5on4_1          0
team_strength_4on5_1          0
xGoalsPercentage_5on5_2       0
team_strength_5on5_2          0
team_strength_5on4_2          0
team_strength_4on5_2          0
goalie_strength_1             0
goalie_strength_2             0
forward_strength_1            0
defense_strength_1            0
forward_strength_2            0
defense_strength_2            0
xGoalsPercentage_last_3_1     0
xGoalsPercentage_last_5_1     0
xGoalsPercentage_last_10_1    0
xGoalsPercentage_last_3_2     0
xGoalsPercentage_last_5_2     0
xGoalsPercentage_last_10_2    0
teams                         0
dtype: int64

In [54]:
input_df[input_df.isnull().any(axis=1)]

,result,home_or_away_t1,home_or_away_t2,xGoalsPercentage_5on5_1,team_strength_5on5_1,team_strength_5on4_1,team_strength_4on5_1,xGoalsPercentage_5on5_2,team_strength_5on5_2,team_strength_5on4_2,...,defense_strength_1,forward_strength_2,defense_strength_2,xGoalsPercentage_last_3_1,xGoalsPercentage_last_5_1,xGoalsPercentage_last_10_1,xGoalsPercentage_last_3_2,xGoalsPercentage_last_5_2,xGoalsPercentage_last_10_2,teams
gameDate,,,,,,,,,,,,,,,,,,,,,
